# Getting Detections using Falconpy and Pandas

## Main Goal: 

`Gathering details from detections directly from the console via API and pivot them for a better visualization`



In [1]:
# Importing libraries
# PS: Not all packages are being used but is good to have them to handle data if necessary
from   falconpy import Detects
from   dateutil   import parser
import pytz
import csv
import os as system
import configparser
import json
import pandas as pd

# Making pandas visualizarion results more attractive
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('max_colwidth', 5000)

# Getting id and secret stored in the hd
# Do not hardcode API credentials!
CONFIGFILE = 'key.ini'
cfg = configparser.ConfigParser()
cfg.read(CONFIGFILE)

# Setting up credentials variables
CLIENT_ID     = cfg.get('CLIENT_ID', 'CLIENT_ID')
CLIENT_SECRET = cfg.get('CLIENT_SECRET', 'CLIENT_SECRET')


## Part 1 - Getting a raw list of detection ids

In [2]:
# Instantiate Detects class
falcon = Detects(client_id=CLIENT_ID,                            
                 client_secret=CLIENT_SECRET,                 
                 )

# Getting the list id of the detections
# Here is a simple filter using date/time of detections
# For more informarion about using filters and FQL go to the documentation page described in README.md

response = falcon.query_detects(limit=9999,                            
                               filter="last_behavior:>='2024-01-02'+last_behavior:<='2024-01-03'+max_severity_displayname:!'Informational'"
                               )

# As a good practice here I'm checking answer from the API before start
if response['status_code'] == 200:    
    id_detections = response['body']['resources']    
    print(len(id_detections))
    
else:
    print(response['body']['errors'])


90


## Part 2 - Gathering detections details

Here I'm using a tricky code to handle over 1k of detections details from [GetDetectSummaries](https://www.falconpy.io/Service-Collections/Detects.html#getdetectsummaries) since this endpoint doesn't have offset and limit parameters

`PS: I'm working on a better solucion using` [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) `and I'll update this script soon


In [3]:
# Making indexes for receive 1k detections details
indexes = [1000,2000,3000,4000,5000,6000,7000,8000,9000,9999]
output = []
prev = 0

# Slicing the id_detections and parts of 1k
for index in indexes:
    output.append(id_detections[prev:index])
    prev = index

# Gathering detection details by 1k
# If you have less detections than 9999 you can make a smaller list of lists
count = -1
full_list = []
while count < len(indexes[:-1]):
    count += 1
    detects       = falcon.get_detect_summaries(ids=output[count]) 
    detail_list   = detects['body']['resources']
    full_list.append(detail_list)  
    
# PS: This will create 10 lists of 1k records for each one. 
len(full_list)


10

## Part 3: Making a big list with all small lists

In [4]:
final_list = []

for i in range(len(full_list)):
    final_list += full_list[i]  
len(final_list)


90

## Part 4: Creating a dataframe with the data collected


You can choose whatever fields you want to compose your dataframe, just check the documentation to get what fields are available

`PS 1: Be carreful if you want to get the IP ADDRESSES because is very common the field in API come absent, at least in my experience`

`PS 2: About action taken (pattern_disposition) I'm using the most common, also considering that the best practices in policy configuration is already applied in you environment`
 
I encourage you the read more in [EventsDataDictionary](https://falcon.crowdstrike.com/documentation/page/e3ce0b24/events-data-dictionary) to have a more wide comprehension of the detections endpoint and how to use it.

In [5]:
# Adjust datetime fields for your country since the API uses UTC as base timestamp 

count_detect = -1
for detection in final_list:
    count_detect += 1
    # Grab the "last_behavior" field
    detect_last_behavior = parser.parse(detection["last_behavior"])
    
    # Create a timestamp in our local timezone
    local_timezone = pytz.timezone("Brazil/East")
    local_datetime = detect_last_behavior.replace(tzinfo=pytz.utc)
    # Convert the UTC date time to a more aware local
    local_datetime = local_datetime.astimezone(local_timezone)
    # Output our debug results    
    final_list[count_detect]['last_behavior'] = str(local_datetime)[:-6]
    

# Creating the dataframe
df = pd.DataFrame(columns=['SEVERITY', 'HOSTNAME', 'TYPE', 'OS', 'LAST_DETECTION', 
                           'USERNAME', 'FILENAME', 'CMD', 'IOC', 'ACTION', 'DESCRIPTION'])

# Populating the dataframe
for i in range(len(final_list)):
    severity       = final_list[i]['max_severity_displayname']
    hostname       = final_list[i]['device']['hostname']
    host_type      = final_list[i]['device']['product_type_desc']
    os             = final_list[i]['device']['os_version']
    last_behavior  = final_list[i]['last_behavior']
    user           = final_list[i]['behaviors'][0]['user_name']    
    filename       = final_list[i]['behaviors'][0]['filename']
    cmdline        = final_list[i]['behaviors'][0]['cmdline']
    ioc            = final_list[i]['behaviors'][0]['ioc_description']
    ptd            = final_list[i]['behaviors'][0]['pattern_disposition']
    description    = final_list[i]['behaviors'][0]['description']
    
    if ptd     == 0:
        action  = 'Detection Only'
    elif ptd   == 16:
        action  = 'Process Killed'
    elif ptd   == 128:
        action  = 'File Quarantined'
    elif ptd   == 272:
        action  = 'Detection Only'
    elif ptd   == 512:
        action  = 'Process Killed'
    elif ptd   == 768:
        action  = 'Detection Only'    
    elif ptd   == 1024:
        action  = 'Operation Blocked'
    elif ptd   == 1280:
        action  = 'Detection Only'
    elif ptd   == 2048:
        action  = 'Process Blocked'
    elif ptd   == 2176:
        action  = 'File Quarantined'
    elif ptd   == 2304:
        action  = 'Detection Only' 
    elif ptd   == 4096:
        action  = 'Registry Operation Blocked'
    elif ptd   == 4112:
        action  = 'Registry Operation Blocked'
    elif ptd   == 4638:
        action  = 'Detection Only' 
    elif ptd   == 32768:
        action  = 'File system operation blocked'
    elif ptd   == 8208:
        action  = 'Process Killed - Parent process killed'
    elif ptd   == 2099200:
        action  = 'Process Blocked - Response action applied'
            
    else:
        action  = 'Cant get action. Check the event'
            
    df.loc[i] = [severity, hostname, host_type, os, last_behavior, user, filename, cmdline, 
                 ioc, action, description]

len(df)


90

# ||   ATTENTION   ||


### Here I'm showing a small sample of the DF just to give an idea that how it will look like, however I needed to hide some data since I haven't a lab environment and all data used is private


In [6]:
df[['SEVERITY', 'TYPE', 'OS', 'LAST_DETECTION', 'FILENAME', 'CMD', 'ACTION', 'DESCRIPTION']].head(2)

,SEVERITY,TYPE,OS,LAST_DETECTION,FILENAME,CMD,ACTION,DESCRIPTION
0,Low,Workstation,Windows 10,2024-01-03 19:26:30,explorer.exe,C:\WINDOWS\Explorer.EXE,File Quarantined,A file written to the file-system was classified as Adware/PUP based on its SHA256 hash.
1,High,Workstation,Windows 10,2024-01-03 18:54:58,WindowsSensor.x64.exe,"""C:\ProgramData\Package Cache\{a5ca1367-4057-471d-aa23-d25568ee038c}\WindowsSensor.x64.exe"" /uninstall",Process Killed - Parent process killed,"A process attempted to uninstall the Falcon sensor in an unusual way. If this is unexpected, it might be an adversary trying to disable the Falcon sensor. Review the process tree."


## Part 5: Adding duplicated values

In [7]:
# Counting duplicate values to have a sum of all detections for specific filename and hostname
count = df.value_counts(['HOSTNAME', 'FILENAME'], dropna=True, sort=True)


## Part 6: Making the `count` object a new DF

It is needed since the the value_counts() returned is a Series, and Series can't be merged with DataFrames

In [8]:
# Converting count Series onto Dataframe
df_count_list = pd.DataFrame(count)

# Reset columns to show in df_count
df_count = df_count_list.reset_index()

# Renaming columns to show accordingly
df_count.columns = ['HOSTNAME', 'FILENAME', 'COUNT']


## Part 7: Sorting by HOSTNAME and LAST_DETECTION 

In [9]:
# Sorting detections by column HOST and LAST_DETECTION
sorted_df = df.sort_values(by=['HOSTNAME', 'LAST_DETECTION'], ascending=False)


## Part 8: Dropping duplicates hostnames

In [10]:
# Removing all unnecessary rows since we already have the full couting of it
df_droped = sorted_df.drop_duplicates(subset=['HOSTNAME', 'FILENAME'], keep="first") 


## Part 9: Merging the two dataframes (with only one last occurrence that host and file)

In [11]:
df_merged = pd.merge(df_droped, df_count, on=['HOSTNAME', 'FILENAME'])


In [12]:
# Showing a sample of the final result
df_merged[['SEVERITY', 'TYPE', 'OS', 'LAST_DETECTION', 'FILENAME', 'ACTION', 'COUNT']].sort_values(by=['COUNT'], ascending=False).head(5)


,SEVERITY,TYPE,OS,LAST_DETECTION,FILENAME,ACTION,COUNT
24,Critical,Workstation,Windows 10,2024-01-03 17:38:03,Utilman.exe,Process Blocked,32
25,High,Workstation,Windows 10,2024-01-03 14:38:47,net.exe,Process Blocked,15
20,Critical,Workstation,Windows 11,2024-01-03 11:03:43,PsExec.exe,File Quarantined,4
4,High,Workstation,Windows 7,2024-01-03 13:12:25,bitsadmin.exe,Process Blocked,4
12,Critical,Server,Windows Server 2016,2024-01-02 10:59:34,PsExec.exe,Process Blocked,3


## Part 10: Exporting the data

In [13]:
file_name = '/home/bruce/PROJECTS/JUPYTER/FALCON/REPORTS/Detections.xlsx'
df_merged.to_excel(file_name)


## Part 11: Optional (quick tip if the detections has some strange chars)

In [ ]:
### *** If raise IllegalCharacterError saving as excel by utf-8 errors, try this *** ###

#file_name = '/home/bruce/PROJECTS/JUPYTER/FALCON/REPORTS/Detections.csv'
#df_merged.to_csv(file_name, encoding='utf-8-sig')
